In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from dLoader import DataLoader, BuySell, generate_df

In [2]:
# Load data
stock_data = DataLoader('AAPL')

In [3]:
# Get a period of data for testing
data = stock_data.get_data('2018-01-01', '2019-12-31')
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,41.188156,102223600
2018-01-03,43.132500,43.637501,42.990002,43.057499,41.180992,118071600
2018-01-04,43.134998,43.367500,43.020000,43.257500,41.372272,89738400
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.843311,94640000
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.687893,82271200
...,...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,70.027313,48478800
2019-12-26,71.205002,72.495003,71.175003,72.477501,71.416672,93121200
2019-12-27,72.779999,73.492500,72.029999,72.449997,71.389572,146266000


In [4]:
def softmax(x):
    # Calculate softmax of x
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [5]:
class MACross:
    # Moving Average Fast Slow Crossover Strategy
    # Check Fast Moving Average against Slow Moving Average
    # Also checking Current Volume Average is above 
    # Volumne Moving Average
    def __init__(self, fast=3, slow=6):
        self.fast=3
        self.slow=6
    
    def check_ma_above(self, df):
        # Create Moving Averages for Price and Volume
        ndf = self.get_ma(df)
        # Check if price and volume is above Moving Averages
        if ndf['MA-Above'].iloc[-1] and ndf['MA-Vol-Above'].iloc[-1]:
            return True
        return False

    def get_ma(self, df):
        # MA Column Creations
        df['Fast'] = df['Close'].rolling(self.fast).mean()
        df['Slow'] = df['Low'].rolling(self.slow).mean()
        df['MA-Vol'] = df['Volume'].rolling(self.slow).mean()
        df['MA-Above'] = df['Fast'] > df['Slow']
        df['MA-Vol-Above'] = df['Volume'] > df['MA-Vol']
        return df


In [6]:
class LookBackTest:
    def __init__(self, fast=3, slow=6, capital=1000, max_share=100):
        # MA Calculation Class
        self.mc = MACross(fast, slow)
        self.lookback = slow + 1
        # Buy Sell
        self.bs = BuySell(capital, max_share)
    
    def recur_lookback(self, data, i=None):
        # Recursion for trading BackTest
        i = self.lookback if i is None else i
        if (i + 1) == len(data):
            self.update_trades(data, i, sell_all=True)
            # Showing Result
            self.bs.show_results()
            return
        # Update trade data
        self.update_trades(data, i)
        # Recursion
        self.recur_lookback(data, i+1)
    
    def update_trades(self, data, idx, sell_all=False):
        # Update record of trades
        date = data.index[idx]
        hPrice, lPrice, cPrice = data.loc[date, ['High', 'Low', 'Close']]
        # Logic for Buying and Selling
        above_ma = self.is_ma_above(data, date)
        if sell_all:
            if self.bs.is_holding:
                self.bs.sell(cPrice)
        else:
            # Buy when not holding
            if not self.bs.is_holding:
                if above_ma:
                    self.bs.buy(lPrice)
            else:
                # Sell when holding
                if not above_ma:
                    self.bs.sell(hPrice)
    
    def is_ma_above(self, data, date):
        # Check if the Price and Volume is above Moving Average
        lookback_data = self.lookback_data(data, date)
        return self.mc.check_ma_above(lookback_data)

    def lookback_data(self, df, current_date):
        # Getting Lookback data base on current date
        lookback_range = pd.date_range(end=current_date, 
                                       periods=self.lookback*2, 
                                       freq='B')
        return df.loc[df.index.isin(lookback_range[: -1])].copy()

In [7]:
lb_test = LookBackTest()

In [8]:
lb_test.recur_lookback(data)

Average gain of $1.19 per share after 82.0 trades
and have average gain percentage of 2.41%
Has a 91.46% of good trades
Test Ending Capital: $6746.04 base on original capital of $1000.00
With 574.60% Capital Gain


In [13]:
# Create different section for possible gain
pdata = stock_data.get_data('2017-01-01', '2017-12-31')
hl = pdata['High'] / pdata['Low'].shift(3) - 1
hlMax = hl.max()
hlMin = hl.min()
hlTotal = hlMax - hlMin
hlRange = np.arange(.00, np.round(hlTotal, 2), .005) + np.round(hlMin, 2)
hlPair = np.stack([hlRange[:-1], hlRange[1:]]).T
dic = {}
for i, pair in enumerate(hlPair):
    isin = (hl >= pair[0]) & (hl < pair[1])
    dic[i] = {'a': pair[0],
              'b': pair[1],
              'Counts': len(hl[isin])}
pct_df = pd.DataFrame.from_dict(dic, orient='index')
# Percentage
pct_df['PCT'] = pct_df['Counts'] / pct_df['Counts'].sum()
# Softmax
pct_df['softmax'] = softmax(pct_df['PCT'])

In [14]:
pct_df.nlargest(10, columns='softmax')

,a,b,Counts,PCT,softmax
12,0.010,0.015,38,0.153846,0.040147
13,0.015,0.020,38,0.153846,0.040147
11,0.005,0.010,29,0.117409,0.038711
15,0.025,0.030,29,0.117409,0.038711
14,0.020,0.025,22,0.089069,0.037629
16,0.030,0.035,19,0.076923,0.037175
9,-0.005,0.000,13,0.052632,0.036282
10,0.000,0.005,13,0.052632,0.036282
7,-0.015,-0.010,7,0.028340,0.035412
8,-0.010,-0.005,6,0.024291,0.035269
